In [33]:
import os
import pandas as pd
import glob

from pathlib import Path

In [34]:
dir_path = Path(os.getcwd())
data_dir = str(os.path.join(dir_path.parent,'data'))
balance_files = glob.glob(data_dir + "/poker_balances*.csv")
hands_files = glob.glob(data_dir + "/poker_hands*.csv")
table_files = glob.glob(data_dir + "/poker_table_info*.csv")

def get_hands_df():
    hands_dfs=[]

    for file in hands_files:
        new_df = pd.read_csv(file)
        hands_dfs.append(new_df)

    hands = pd.concat(hands_dfs)
    return hands
    
def get_games_df():
    balance_dfs=[]
    
    for file in balance_files:
        new_df = pd.read_csv(file)
        balance_dfs.append(new_df)
        
    balances = pd.concat(balance_dfs)
    return balances

def get_table_df():
    table_dfs=[]
    
    for file in table_files:
        new_df = pd.read_csv(file)
        table_dfs.append(new_df)
        
    tables = pd.concat(table_dfs)
    return tables


In [35]:
pd.set_option('display.max_columns', None)  
hands_base = get_hands_df()
games_base = get_games_df()
tables = get_table_df()

games = pd.merge(games_base,tables)
hands = pd.merge(hands_base,games)

In [36]:
tables

,table_id,scenario_name,player_types
0,1,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
0,10,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
0,11,all_raise_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
0,12,all_raise_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
0,13,all_raise_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
0,14,all_raise_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
0,15,all_raise_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
0,16,all_raise_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
0,17,all_raise_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
0,18,all_raise_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...


In [37]:
games

,table_id,game_id,player_name,player_type,game_result,game_reason,blind_type,beginning_balance,game_start_balance,game_end_balance,game_net_change,scenario_name,player_types
0,1,1,players_3,AlwaysCallPlayer,won,won_game,None,100000,100000,100000,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
1,1,2,players_3,AlwaysCallPlayer,won,won_game,None,100000,100000,100000,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
2,1,3,players_3,AlwaysCallPlayer,won,won_game,small,100000,100000,100000,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
3,1,4,players_3,AlwaysCallPlayer,won,won_game,big,100000,100000,100000,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
4,1,5,players_3,AlwaysCallPlayer,won,won_game,None,100000,100000,100000,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,9,896,players_2,AlwaysCallPlayer,won,won_game,None,100000,100030,100030,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
11996,9,897,players_2,AlwaysCallPlayer,won,won_game,None,100000,100030,100030,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
11997,9,898,players_2,AlwaysCallPlayer,won,won_game,None,100000,100030,100030,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
11998,9,899,players_2,AlwaysCallPlayer,won,won_game,None,100000,100030,100030,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...


In [38]:
hands

,table_id,game_id,player_name,player_type,bet_number,opponents,call,current,final,pot,allowed,hand1,hand2,community1,community2,community3,community4,community5,game_result,game_reason,blind_type,beginning_balance,game_start_balance,game_end_balance,game_net_change,scenario_name,player_types
0,1,1,players_3,AlwaysCallPlayer,1,5,10,0,10.0,35,True,7-spades,K-spades,Z-N/A,Z-N/A,Z-N/A,Z-N/A,Z-N/A,won,won_game,None,100000,100000,100000,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
1,1,1,players_3,AlwaysCallPlayer,2,5,10,10,20.0,90,True,7-spades,K-spades,Z-N/A,Z-N/A,Z-N/A,Z-N/A,Z-N/A,won,won_game,None,100000,100000,100000,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
2,1,1,players_3,AlwaysCallPlayer,3,5,10,20,30.0,150,False,7-spades,K-spades,Z-N/A,Z-N/A,Z-N/A,Z-N/A,Z-N/A,won,won_game,None,100000,100000,100000,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
3,1,1,players_3,AlwaysCallPlayer,4,5,0,30,30.0,180,True,7-spades,K-spades,4-hearts,A-clubs,A-spades,Z-N/A,Z-N/A,won,won_game,None,100000,100000,100000,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
4,1,1,players_3,AlwaysCallPlayer,5,5,10,30,40.0,210,True,7-spades,K-spades,4-hearts,5-spades,A-clubs,A-spades,Z-N/A,won,won_game,None,100000,100000,100000,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81663,9,900,players_2,AlwaysCallPlayer,3,5,10,10,20.0,80,True,7-hearts,A-clubs,6-diamonds,7-spades,J-diamonds,Z-N/A,Z-N/A,won,won_game,small,100000,100030,100030,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
81664,9,900,players_2,AlwaysCallPlayer,4,5,10,20,30.0,140,False,7-hearts,A-clubs,6-diamonds,7-spades,J-diamonds,Z-N/A,Z-N/A,won,won_game,small,100000,100030,100030,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
81665,9,900,players_2,AlwaysCallPlayer,5,5,0,30,30.0,180,True,7-hearts,A-clubs,6-diamonds,7-spades,9-clubs,J-diamonds,Z-N/A,won,won_game,small,100000,100030,100030,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
81666,9,900,players_2,AlwaysCallPlayer,6,5,0,30,30.0,180,True,7-hearts,A-clubs,2-hearts,6-diamonds,7-spades,9-clubs,J-diamonds,won,won_game,small,100000,100030,100030,0,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...


# Basic Game Statistics

Number of Games played in the simulation

In [39]:
games = games[games['scenario_name'] == 'all_raise_against_smart_player']

In [40]:
number_of_games = len(set(games.game_id))
number_of_games

1000

Players Mean number of chips gained and variance

In [41]:
chips_gained_per_game = games.groupby('player_name').mean('game_net_change')[['game_net_change']].reset_index()
variance_calculation = games[['player_name','game_net_change']]
variance_calculation['game_net_change'] = variance_calculation['game_net_change'] ** 2
expectation_squared = variance_calculation.groupby('player_name').mean('game_net_change').reset_index()
merged_expectations = pd.merge(expectation_squared,chips_gained_per_game,on='player_name')
merged_expectations['variance'] = merged_expectations['game_net_change_x'] - merged_expectations['game_net_change_y']
summary_stats = merged_expectations[['player_name','game_net_change_y','variance']]
summary_stats.columns = ['player_name','mean','variance']
summary_stats['games'] = number_of_games
summary_stats

c:\users\silbe\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,player_name,mean,variance,games
0,players_1,0.363,2.072,1000
1,players_2,0.363,2.072,1000
2,players_3,0.363,2.072,1000
3,players_4,0.363,2.072,1000
4,players_5,0.363,2.072,1000
5,players_6,-1.815,62.690,1000


Top 10 Best Card Hands Dealt by Mean Number of Chips Gained 

In [42]:
two_card_hands = hands[['game_id','player_name','player_type','hand1','hand2','game_net_change']]
two_card_hands = two_card_hands.drop_duplicates()
best_two_card_hands = two_card_hands.groupby(['player_name','hand1','hand2']).mean().reset_index()[['player_name','hand1','hand2','game_net_change']]
best_two_card_hands = best_two_card_hands.sort_values(by=['player_name','game_net_change'], ascending=[True,False])
best_two_card_hands.groupby('player_name').head(10)

,player_name,hand1,hand2,game_net_change
55,players_1,10-diamonds,9-clubs,8.0
418,players_1,4-clubs,8-hearts,8.0
450,players_1,4-diamonds,8-hearts,8.0
688,players_1,6-hearts,8-spades,8.0
814,players_1,8-clubs,A-diamonds,8.0
637,players_1,6-clubs,8-hearts,7.0
120,players_1,10-spades,6-hearts,6.0
308,players_1,3-clubs,J-hearts,6.0
525,players_1,4-spades,Q-clubs,6.0
702,players_1,6-hearts,Q-diamonds,6.0
